In [ ]:
from dotenv import load_dotenv
from elasticsearch import Elasticsearch
from langchain.document_loaders import DataFrameLoader
import loggingService
import os
import pandas as pd
import requests
from sentence_transformers import SentenceTransformer

load_dotenv()
logger = loggingService.get_logger()


In [ ]:
# Declare a client instance of the Python Elasticsearch library.
elastic_url = os.getenv("ELASTICSEARCH_URL", "http://127.0.0.1:9200")
index_name = os.getenv("INDEX_NAME", 'vector_index')
api_url = f"{elastic_url}/{index_name}/"

client = Elasticsearch(hosts=[elastic_url])


In [29]:
df = pd.read_csv('./data/listaPlanosTIMPrd.csv', index_col=0)
df.head()

df.drop(['updatedBy', 'createdBy', 'createdAt', 'updatedAt', 'renewalText'], axis='columns', inplace=True)
df = df.fillna(0)
df = df.astype(str)
# df.to_csv('./data/planosTIMClean.csv')
#    'mecanism', 'roamingInternacional'
df['text'] = df['plan'] + ' ' + df['description'] + ' ' + df['dataFranchise'] + ' ' + df['voiceFranchise'] + ' ' + df['smsFranchise'] + ' ' + df['socialNetworkFranchise'] + ' ' + df['mecanism'] + ' ' + df['roamingInternacional']

df.head()
df.describe()

loader = DataFrameLoader(df, page_content_column="text")
data = loader.load()


In [33]:
print(data[0])
df.columns

page_content='TIM CONTROLE GIGA C PROMO Com o plano, TIM Controle, GIGA C,PROMO . Você tem ligações ilimitadas,para qualquer operadora usando, o 41e ainda pode enviar SMS ilimitado.  E usar os aplicativos: WhatsApp, Messenger e Telegram ,sem descontar nada da sua franquia de dados. Além de ter, 4.5 Giga de internet para usar como quiser. Contando com o seu plano, e com seus pacotes de internet, você tem #totalDados#, no total. E ainda tem disponível #totalDisponivel# de Internet pra usar como quiser! Com o seu plano, você tem ligações ilimitadas para qualquer operadora, usando o 41. Você tem, SMS ilimitado para TIM, e para outras operadoras. Com o seu plano, você tem o aplicativo Waze para usar à vontade.  Além do Messenger, Telegram, e WhatsApp, para enviar e receber mensagens sem descontar da sua franquia de dados.  E tem mais! Você também conta com TIM Music. TIM protect bé cá pi 5 giga, e TIM banca virtual, incluídos. 0 0' metadata={'commercialCode': 'PLN586', 'plan': 'TIM CONTROLE

Index(['commercialCode', 'plan', 'description', 'dataFranchise',
       'voiceFranchise', 'smsFranchise', 'socialNetworkFranchise', 'mecanism',
       'segment', 'upsellDescription', 'planValues', 'status',
       'roamingInternacional', 'text'],
      dtype='object')

In [30]:
def createSchema():
    schema = {
        "mappings": {
            "properties": {
                "embedding": {
                    "type": "dense_vector",
                    "dims": 1024,
                    "index": True,
                    "similarity": "cosine"
                },
                "commercialCode": {"type": "text"}, 
                "plan": {"type": "text"},
                "description": {"type": "text"},
                "dataFranchise": {"type": "text"},
                "voiceFranchise": {"type": "text"},
                "smsFranchise": {"type": "text"},
                "socialNetworkFranchise": {"type": "text"},
                "mecanism": {"type": "text"},
                "segment": {"type": "text"},
                "upsellDescription": {"type": "text"},
                "planValues": {"type": "text"},
                "status": {"type": "text"},
                "roamingInternacional": {"type": "text"},
                "text": {"type": "text"}
            }
        }
    }

    response = requests.put(api_url, json=schema)

    logger.info(response.json())
    logger.info(response.status_code)